#Lab1
##0-1Knapsack

In [12]:
import numpy as np

###Thanks to professor

In [13]:
NUM_KNAPSACKS = 3
NUM_ITEMS = 10
NUM_DIMENSIONS = 2

In [14]:
VALUES = np.random.randint(0, 100, size=NUM_ITEMS)
WEIGHTS = np.random.randint(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = np.random.randint(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=(NUM_KNAPSACKS,NUM_DIMENSIONS))

In [15]:
print(VALUES)
print(WEIGHTS)
CONSTRAINTS

[85 96 73 14 97 90 94 83 95 82]
[[57 50]
 [12 99]
 [30 41]
 [29 79]
 [13 43]
 [20 99]
 [48 35]
 [95  4]
 [19 79]
 [23 86]]


array([[183,  37],
       [172,  96],
       [282, 288]])

In [16]:
#test some solutions
#start from classic knapsack solution
#贪心算法
def greedy_solution(values, weights, constraints, num_knapsacks, *, eps=1e-9):
    
    values = np.asarray(values)
    weights = np.asarray(weights)
    constraints = np.asarray(constraints)

    n_items, n_dims = weights.shape
    sol = np.zeros((n_items, num_knapsacks), dtype=int)
    used = np.zeros((num_knapsacks, n_dims), dtype=int)

    norm = np.maximum(constraints.astype(float), 1.0)
    cost = (weights / norm).sum(axis=1) + eps
    order = np.argsort(-(values / cost))  

    for i in order:
        best = None
        for k in range(num_knapsacks):
            if np.all(used[k] + weights[i] <= constraints):
                residual_sum = (constraints - (used[k] + weights[i])).sum()
                if (best is None) or (residual_sum < best[0]):
                    best = (residual_sum, k)
        if best is not None:
            k = best[1]
            sol[i, k] = 1
            used[k] += weights[i]
    return sol

#check feasibility
def check_solution(solution, weights, constraints):
    
    S = np.asarray(solution)
    W = np.asarray(weights)
    C = np.asarray(constraints)

    # shape checks
    if S.ndim != 2 or W.ndim != 2 or C.ndim != 1:
        return False
    if S.shape[0] != W.shape[0] or W.shape[1] != C.shape[0]:
        return False

    # 0/1 and one-hot checks
    if not np.all((S == 0) | (S == 1)):
        return False
    if not np.all(S.sum(axis=1) <= 1):
        return False

    # capacity check
    used = S.T @ W  # (n_knapsacks, n_dims)
    if np.any(used > C):
        return False

    return True


In [17]:
# Check if the solution is valid
all_knapsacks = np.any(greedy_solution, axis=0)
#np.all(WEIGHTS[all_knapsacks].sum(axis=0) < CONSTRAINTS)

###TEST PROBLEMS

In [18]:
# Problem 1:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 3
NUM_ITEMS = 20
NUM_DIMENSIONS = 2
VALUES = rng.integers(0, 100, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 100, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(0, 100 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

sol = greedy_solution(VALUES, WEIGHTS, CONSTRAINTS, NUM_KNAPSACKS)
print(sol)



che = check_solution(sol, WEIGHTS, CONSTRAINTS)
print(che)
#try fitness function


[[0 0 1]
 [1 0 0]
 [0 1 0]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]]
True


In [19]:
# Problem 2:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 10
NUM_ITEMS = 100
NUM_DIMENSIONS = 10
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 2, 1000 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)

greedy_solution = np.zeros((NUM_ITEMS, NUM_KNAPSACKS), dtype=int)
item_indices = np.argsort(-VALUES / np.sum(WEIGHTS, axis=1))    
knapsack_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
for idx in item_indices:
    for k in range(NUM_KNAPSACKS):
        if np.all(knapsack_weights[k] + WEIGHTS[idx] <= CONSTRAINTS):
            greedy_solution[idx, k] = 1
            knapsack_weights[k] += WEIGHTS[idx]
            break
print(greedy_solution)

#check feasibility
def is_feasible(solution):
    knapsack_weights = np.zeros((NUM_KNAPSACKS, NUM_DIMENSIONS), dtype=int)
    for i in range(NUM_ITEMS):
        for k in range(NUM_KNAPSACKS):
            if solution[i, k] == 1:
                knapsack_weights[k] += WEIGHTS[i]
    return np.all(knapsack_weights <= CONSTRAINTS)
is_feasible(greedy_solution)

[[0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 1 0 0 0]
 [0 0 0 0 1

np.True_

In [20]:
# Problem 3:
rng = np.random.default_rng(seed=42)
NUM_KNAPSACKS = 100
NUM_ITEMS = 5000
NUM_DIMENSIONS = 100
VALUES = rng.integers(0, 1000, size=NUM_ITEMS)
WEIGHTS = rng.integers(0, 1000, size=(NUM_ITEMS, NUM_DIMENSIONS))
CONSTRAINTS = rng.integers(1000 * 10, 1000 * 2 * NUM_ITEMS // NUM_KNAPSACKS, size=NUM_DIMENSIONS)


In [21]:
print(VALUES)
print(WEIGHTS)
print(CONSTRAINTS)

[ 89 773 654 ... 819 678 498]
[[197  33 627 ... 319 937 293]
 [550 543 242 ... 942 568 950]
 [436 411 836 ... 664 169 846]
 ...
 [869 582 968 ... 567 291 288]
 [ 97 220 662 ... 171 310 833]
 [656  51 146 ... 286 725 372]]
[79834 52005 17527 30007 59295 98921 12881 36174 63958 90566 25740 12251
 27746 78136 90704 39249 23072 69010 17326 29277 42886 48016 74931 93479
 30898 18137 75557 56030 75506 90246 42093 34668 22075 79709 98971 15032
 57157 21369 76865 11589 82666 78074 68715 85558 61242 80943 26905 84378
 85069 77221 16434 28156 17827 83548 72921 27512 41395 23936 72519 96465
 12904 72709 81023 81015 28959 98041 90482 89802 76408 29899 51171 94767
 78351 57860 58504 40545 97025 99185 42888 54965 24945 51281 31276 61913
 96714 93034 72901 95222 37857 53461 15463 71355 64979 70086 48286 38026
 60109 78500 56643 69915]


In [22]:
rng = np.random.default_rng(0)
NUM_KNAPSACKS = 2
NUM_ITEMS = 3
NUM_DIMENSIONS = 2

VALUES   = rng.integers(0, 10, size=NUM_ITEMS)              # (3,)
WEIGHTS  = rng.integers(0, 10, size=(NUM_ITEMS, NUM_DIMENSIONS))  # (3,2)
# 单背包/共享容量（不推荐多背包用）
CONSTRAINTS_shared = rng.integers(5, 15, size=NUM_DIMENSIONS)      # (2,)
# 多背包容量（推荐多背包用）
CONSTRAINTS_multi  = rng.integers(5, 15, size=(NUM_KNAPSACKS, NUM_DIMENSIONS))  # (2,2)

print("VALUES.shape:", VALUES.shape)            # (3,)
print("WEIGHTS.shape:", WEIGHTS.shape)          # (3,2)
print("共享容量:", CONSTRAINTS_shared.shape)     # (2,)
print("多背包容量:", CONSTRAINTS_multi.shape)    # (2,2)

print("VALUES =", VALUES)
print("WEIGHTS=\n", WEIGHTS)
print("CONSTRAINTS_multi=\n", CONSTRAINTS_multi)


VALUES.shape: (3,)
WEIGHTS.shape: (3, 2)
共享容量: (2,)
多背包容量: (2, 2)
VALUES = [8 6 5]
WEIGHTS=
 [[2 3]
 [0 0]
 [0 1]]
CONSTRAINTS_multi=
 [[14 10]
 [11 14]]
